# Azure AI Agents with Model Context Protocol (MCP) Support

ဒီ notebook က Azure AI Agents ကို Model Context Protocol (MCP) tools တွေနဲ့အတူ အသုံးပြုပြီး အပြင် MCP servers တွေကို အသုံးချနိုင်တဲ့ အတတ်ပညာရှိ Agent တစ်ခု ဖန်တီးပုံကို ပြသပေးမှာ ဖြစ်ပါတယ်။


## လိုအပ်သော NuGet Packages ကို ထည့်သွင်းပါ

ပထမဦးဆုံး၊ Azure AI Agents နှင့်အလုပ်လုပ်ရန် အခြေခံလုပ်ဆောင်ချက်များပေးသော Azure AI Agents Persistent package ကို ထည့်သွင်းရန်လိုအပ်ပါသည်။


## Keyless Authentication အကျိုးကျေးဇူးများ

ဒီ notebook က **keyless authentication** ကို ပြသပေးမှာဖြစ်ပြီး အောက်ပါ အကျိုးကျေးဇူးများကို ပေးစွမ်းနိုင်ပါတယ်။
- ✅ **API key မလိုအပ်ခြင်း** - Azure identity-based authentication ကို အသုံးပြုခြင်း
- ✅ **လုံခြုံရေး မြှင့်တင်ခြင်း** - Code သို့မဟုတ် configuration မှာ secret မရှိခြင်း
- ✅ **အလိုအလျောက် credential ပြောင်းလဲမှု** - Azure က credential lifecycle ကို စီမံပေးခြင်း
- ✅ **Role-based access** - Azure RBAC ကို အသုံးပြုပြီး ခွင့်ပြုချက်များကို စိတ်ကြိုက် သတ်မှတ်နိုင်ခြင်း

`DefaultAzureCredential` က အကောင်းဆုံး credential အရင်းအမြစ်ကို အလိုအလျောက် အသုံးပြုပါမယ်။
1. Managed Identity (Azure မှာ run လုပ်နေတဲ့အခါ)
2. Azure CLI credentials (ဖွံ့ဖြိုးရေးအဆင့်မှာ)
3. Visual Studio credentials
4. Environment variables (configure လုပ်ထားပါက)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## လိုအပ်သော နေရာအမည်များကို သွင်းယူပါ

Azure AI Agents နှင့် Azure Identity အတွက် လိုအပ်သော နေရာအမည်များကို သွင်းယူပါ။


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Client (Keyless Authentication) ကိုပြင်ဆင်ခြင်း

**keyless authentication** ကို အသုံးပြု၍ configuration variables များကို ပြင်ဆင်ပြီး PersistentAgentsClient ကို ဖန်တီးပါ။
- **projectEndpoint**: Azure AI Foundry project endpoint
- **modelDeploymentName**: တင်သွင်းထားသော AI မော်ဒယ် (GPT-4.1 nano) ၏ အမည်
- **mcpServerUrl**: MCP server (Microsoft Learn API) ၏ URL
- **mcpServerLabel**: MCP server ကို သတ်မှတ်ရန် အသုံးပြုသော label
- **DefaultAzureCredential**: managed identity, Azure CLI, သို့မဟုတ် အခြားသော credential sources များကို အသုံးပြုသည် (API key မလိုအပ်ပါ)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP Tool ကိုသတ်မှတ်ခြင်း

Microsoft Learn MCP server ကိုချိတ်ဆက်နိုင်သော MCP tool ကိုသတ်မှတ်ပါ။ ဒါကအေးဂျင့်ကို Microsoft Learn အကြောင်းအရာနှင့်စာရွက်စာတမ်းများကိုရယူနိုင်စေပါမည်။


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI ကိုယ်စားလှယ်တစ်ဦး ဖန်တီးရန်

သတ်မှတ်ထားသော မော်ဒယ်နှင့် MCP ကိရိယာများဖြင့် အမြဲတမ်း AI ကိုယ်စားလှယ်တစ်ဦး ဖန်တီးပါ။ အဆိုပါ ကိုယ်စားလှယ်ကို အောက်ပါအတိုင်း ပြင်ဆင်ထားသည် -  
- GPT-4.1 nano မော်ဒယ်  
- MCP ကိရိယာများကို အကူအညီရယူရန် အသုံးပြုရန် ညွှန်ကြားချက်များ  
- Microsoft Learn MCP server သို့ ဝင်ရောက်ခွင့်


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Thread တစ်ခုဖန်တီးပြီး Message ပို့ရန်

Azure OpenAI နဲ့ OpenAI အကြားကွာခြားချက်ကို မေးမြန်းတဲ့ user message ကိုပို့ပြီး စကားဝိုင်း thread တစ်ခုဖန်တီးပါ။ ဒါက agent ရဲ့ MCP tools ကို အသုံးပြုပြီး တိကျတဲ့ အချက်အလက်ပေးနိုင်စွမ်းကို စမ်းသပ်မှာဖြစ်ပါတယ်။


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP Tool အရင်းအမြစ်များကို ပြင်ဆင်ခြင်း (Keyless)

MCP tool အရင်းအမြစ်များကို ပြင်ဆင်ပါ။ အမှန်တကယ် keyless နည်းလမ်းအတွက် MCP server သည် Azure identity-based authentication ကို ပံ့ပိုးနိုင်ပါက custom headers မလိုအပ်တော့ပါ။ အောက်တွင် ပုံမှန် headers များကို ထည့်သွင်းပုံကို ပြထားပြီး၊ keyless နည်းလမ်းများအတွက်တော့ အဲဒီ headers မလိုအပ်နိုင်ပါ။


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## အေးဂျင့်လုပ်ဆောင်မှု စတင်ရန်

အသုံးပြုသူ၏ မက်ဆေ့ချ်ကို လုပ်ဆောင်ရန်အတွက် လုပ်ဆောင်မှုတစ်ခုကို ဖန်တီးပြီး စတင်ပါ။ အေးဂျင့်သည် တပ်ဆင်ထားသော MCP ကိရိယာများနှင့် အရင်းအမြစ်များကို အသုံးပြု၍ တုံ့ပြန်မှုကို ဖန်တီးမည်။


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## အလုပ်လည်ပတ်မှုကိုကြည့်ရှုပြီး Tool အတည်ပြုမှုများကို ကိုင်တွယ်ခြင်း (Keyless)

Agent အလုပ်လည်ပတ်မှုအခြေအနေကိုကြည့်ရှုပြီး လိုအပ်သော Tool အတည်ပြုမှုများကို ကိုင်တွယ်ပါ။ ဒီ loop:
1. အလုပ်လည်ပတ်မှုပြီးဆုံးရန် သို့မဟုတ် လိုအပ်သော လုပ်ဆောင်မှုကို စောင့်ဆိုင်းသည်
2. MCP tool ကိုခေါ်ဆိုမှုများ လိုအပ်သောအခါ အလိုအလျောက် အတည်ပြုသည်
3. Keyless authentication အတွက် MCP server သည် Azure identity ကို ပံ့ပိုးပါက headers မလိုအပ်နိုင်


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## စကားဝိုင်းရလဒ်များ ပြသရန်

Thread အတွင်းရှိ မက်ဆေ့များအားလုံးကို ပြန်လည်ရယူပြီး ပြသပါ၊ အသုံးပြုသူ၏ မေးခွန်းနှင့် အေးဂျင့်၏ အဖြေတို့ကိုလည်း ဖော်ပြပါ။ မက်ဆေ့များကို အချိန်အလိုက် အစဉ်လိုက်ပြသပြီး Timestamp နှင့် Role အချက်ပြများပါဝင်သည်။


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူရင်းဘာသာစကားဖြင့် အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားယူမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
